## [Step 1] 환경 설정 및 라이브러리 설치

In [ ]:
# 1. 필수 라이브러리 설치
!pip install segmentation-models-pytorch albumentations rasterio shapely geopandas koreanize_matplotlib

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')

In [ ]:
import os
import cv2
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import segmentation_models_pytorch as smp
import albumentations as A
from shapely.geometry import Polygon
from shapely.validation import make_valid
import rasterio.features
from skimage.morphology import skeletonize, closing, square
from scipy.ndimage import distance_transform_edt
from collections import deque

In [ ]:
# A100용 설정
CONFIG = {
    # 1. 경로 설정 (사용자 제공 경로 반영)
    "ROOT": "/content/drive/MyDrive/Colab Notebooks/Data/temp_extract",

    # 2. 학습 설정
    "IMG_SIZE": 1024,   # 원본 해상도 학습 (A100 권장)
    "BATCH_SIZE": 4,    # A100 기준 4~8 (OOM 발생시 2로 줄임)
    "EPOCHS": 20,       # 충분한 학습
    "LR": 0.001,
    "DEVICE": 'cuda' if torch.cuda.is_available() else 'cpu'
}

print(f"🔥 현재 장치: {CONFIG['DEVICE']} (A100 감지됨)" if torch.cuda.is_available() else "⚠️ CPU 모드입니다.")

## [Step 2] 만능 데이터셋 클래스 (도로/건물 공용)

##### 이 클래스는 target_type이라는 옵션을 받아서, 도로를 학습할 때는 도로 라벨을, 건물을 학습할 때는 건물 라벨을 자동으로 찾아옵니다.

In [ ]:
class UnifiedSatelliteDataset(Dataset):
    def __init__(self, img_dir, lbl_dir, transform=None, target_type='road'):
        """
        target_type: 'road' 또는 'building'
        """
        self.img_dir = img_dir
        self.lbl_dir = lbl_dir
        self.transform = transform
        self.target_type = target_type

        # JSON 키 설정 (도로용 vs 건물용)
        if target_type == 'road':
            self.json_key = 'road_imcoords'
        else:
            self.json_key = 'building_imcoords'

        # 이미지 목록 로드
        exts = {'.tif', '.tiff', '.jpg', '.png'}
        self.all_imgs = [f for f in sorted(os.listdir(img_dir)) if os.path.splitext(f)[1].lower() in exts and not f.startswith('.')]
        self.label_map = {f: f for f in os.listdir(lbl_dir) if not f.startswith('.')}

        # 유효 데이터 필터링 (라벨이 있는 데이터만 학습)
        self.valid_imgs = []
        print(f"🔍 [{target_type.upper()}] 데이터 필터링 중...")
        for img in tqdm(self.all_imgs):
            if self.check_label_exists(img):
                self.valid_imgs.append(img)
        print(f"✅ {len(self.valid_imgs)}장 준비 완료")

    def __len__(self):
        return len(self.valid_imgs)

    def find_json(self, img_name):
        base = os.path.splitext(img_name)[0]
        json_name = base + '.json'
        # PS4 -> PS3 이름 변환 대응
        if 'PS4' in base:
            alt_name = base.replace('PS4', 'PS3') + '.json'
            if alt_name in self.label_map: return os.path.join(self.lbl_dir, alt_name)

        if json_name in self.label_map: return os.path.join(self.lbl_dir, json_name)
        return None

    def check_label_exists(self, img_name):
        json_path = self.find_json(img_name)
        if not json_path: return False
        try:
            with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
            for feat in data.get('features', []):
                val = feat['properties'].get(self.json_key, '')
                if val and val != "EMPTY": return True
        except: pass
        return False

    def create_mask(self, json_path, shape):
        mask = np.zeros((shape[0], shape[1]), dtype=np.uint8)
        if not json_path: return mask.astype('float32')
        try:
            with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
            polys = []
            for feat in data.get('features', []):
                val = feat['properties'].get(self.json_key, '')
                if val and val != "EMPTY":
                    coords = [float(x) for x in val.split(',')]
                    pts = list(zip(coords[0::2], coords[1::2]))
                    if len(pts)>=3:
                        p = Polygon(pts)
                        if not p.is_valid: p = make_valid(p)
                        if p.geom_type=='MultiPolygon': polys.extend(p.geoms)
                        else: polys.append(p)
            if polys:
                rasterio.features.rasterize(polys, out=mask, default_value=1, transform=rasterio.Affine(1.0,0,0,0,1.0,0), dtype='uint8')
        except: pass
        return mask.astype('float32')

    def __getitem__(self, idx):
        img_name = self.valid_imgs[idx]
        img_path = os.path.join(self.img_dir, img_name)

        img = cv2.imread(img_path)
        if img is None: return torch.zeros((3,1024,1024)), torch.zeros((1,1024,1024))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        json_path = self.find_json(img_name)
        mask = self.create_mask(json_path, img.shape)

        if self.transform:
            aug = self.transform(image=img, mask=mask)
            img, mask = aug['image'], aug['mask']

        return torch.tensor(img.transpose(2,0,1)/255.0, dtype=torch.float32), torch.tensor(np.expand_dims(mask, 0), dtype=torch.float32)

## [Step 3] 학습 실행 (도로 모델 -> 건물 모델 순차 실행)
##### 이 코드를 실행하면 두 개의 모델 파일이 생성됩니다. (버전 수정 필요)

 1. best_road_model_버전.pth

 2. best_building_model_버전.pth

In [ ]:
# def train_specific_model(target_type, save_name):
#     print(f"\n🚀 [학습 시작] 목표: {target_type.upper()} 모델 생성")

#     # 1. 경로 설정 (자동 매핑)
#     if target_type == 'road':
#         train_img = os.path.join(CONFIG['ROOT'], 'train_roads_images')
#         train_lbl = os.path.join(CONFIG['ROOT'], 'train_roads_labels')
#         val_img = os.path.join(CONFIG['ROOT'], 'validate_roads_images')
#         val_lbl = os.path.join(CONFIG['ROOT'], 'validate_roads_labels')
#     else: # building
#         train_img = os.path.join(CONFIG['ROOT'], 'train_buildings_images')
#         train_lbl = os.path.join(CONFIG['ROOT'], 'train_building_labels') # 폴더명 주의 (buildings -> building)
#         val_img = os.path.join(CONFIG['ROOT'], 'validate_buildings_images')
#         val_lbl = os.path.join(CONFIG['ROOT'], 'validate_buildings_labels')

#     # 2. 데이터셋 & 로더
#     tf = A.Compose([A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']), A.HorizontalFlip(p=0.5), A.RandomRotate90(p=0.5)])
#     v_tf = A.Compose([A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'])])

#     train_ds = UnifiedSatelliteDataset(train_img, train_lbl, transform=tf, target_type=target_type)
#     val_ds = UnifiedSatelliteDataset(val_img, val_lbl, transform=v_tf, target_type=target_type)

#     if len(train_ds) == 0:
#         print(f"❌ {target_type} 데이터가 없습니다. 경로를 확인하세요.")
#         return

#     train_loader = DataLoader(train_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=4)
#     val_loader = DataLoader(val_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=4)

#     # 3. 모델 정의
#     model = smp.Unet(encoder_name="efficientnet-b0", in_channels=3, classes=1, activation='sigmoid').to(CONFIG['DEVICE'])
#     loss_fn = smp.losses.TverskyLoss(mode='binary', alpha=0.3, beta=0.7)
#     optimizer = optim.Adam(model.parameters(), lr=CONFIG['LR'])

#     # 4. 학습 루프
#     best_loss = float('inf')
#     save_path = f"/content/drive/MyDrive/Colab Notebooks/Data/{save_name}"

#     for epoch in range(CONFIG['EPOCHS']):
#         model.train()
#         loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
#         for img, msk in loop:
#             img, msk = img.to(CONFIG['DEVICE']), msk.to(CONFIG['DEVICE'])
#             optimizer.zero_grad()
#             loss = loss_fn(model(img), msk)
#             loss.backward()
#             optimizer.step()
#             loop.set_postfix(loss=loss.item())

#         # 검증
#         model.eval()
#         v_loss = 0
#         with torch.no_grad():
#             for img, msk in val_loader:
#                 img, msk = img.to(CONFIG['DEVICE']), msk.to(CONFIG['DEVICE'])
#                 v_loss += loss_fn(model(img), msk).item()

#         avg_v = v_loss / len(val_loader)
#         print(f"📉 Val Loss: {avg_v:.4f}")

#         if avg_v < best_loss:
#             best_loss = avg_v
#             torch.save(model.state_dict(), save_path)
#             print(f"💾 최고 성능 갱신 -> 저장됨: {save_name}")

# # --- 실행부 ---
# # 1. 도로 모델 학습
# train_specific_model('road', 'best_road_model_1024.pth')

# # 2. 건물 모델 학습
# train_specific_model('building', 'best_building_model_1024.pth')

## [Step 4] 통합 기동 분석 시스템 (도로 + 건물 + 경로 탐색)
##### 이제 학습된 두 모델을 모두 사용하여 "도로와 건물을 동시에 고려하는" 최종 시스템입니다.

* 기능:
   1. 도로 모델: 기동로(초록/빨강) 분석

   2. 건물 모델: 건물 위치(파란색) 시각화 -> 작전 시 엄폐물/장애물 정보 제공

   3. 경로 탐색: 최적 경로 산출

이 코드를 실행하고 이미지 업로드 -> 장비 선택 -> 클릭하면 됩니다.

In [ ]:
import torch
import numpy as np
import cv2
import io
import base64
import os
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from google.colab import files
from google.colab.output import eval_js
from IPython.display import display, HTML, clear_output
from PIL import Image
from skimage.morphology import closing, square, dilation, disk
from scipy.ndimage import distance_transform_edt
from skimage.graph import MCP_Geometric

from IPython.display import HTML



# 1. 파일 로더
def load_image_robust():
    print("\n📤 분석할 위성사진을 업로드하세요.")
    uploaded = files.upload()
    if not uploaded: return None, None
    filename = list(uploaded.keys())[0]
    with open(filename, "wb") as f: f.write(uploaded[filename])
    img = cv2.imread(filename)
    if img is None:
        try:
            pil_img = Image.open(filename).convert('RGB')
            img = np.array(pil_img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        except: return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(f"✅ 로드 완료: {filename} ({img.shape[1]}x{img.shape[0]})")
    return img, filename

# 2. 통합 시스템 클래스
class CommandCenterSystem:
    def __init__(self, road_pth, bldg_pth):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.road_model = self._load_model(road_pth)
        self.bldg_model = self._load_model(bldg_pth)
        self.vehicles = {"K2 흑표": 3.6, "K9 자주포": 3.4, "K808 장갑차": 2.7, "소형전술차량": 2.2, "두돈반": 2.4}
        if self.road_model and self.bldg_model:
            print("✅ 전술 지휘 시스템 가동 (동적 장애물 모듈 탑재)")

    def _load_model(self, path):
        model = smp.Unet(encoder_name="efficientnet-b0", in_channels=3, classes=1, activation='sigmoid').to(self.device)
        if path and os.path.exists(path):
            model.load_state_dict(torch.load(path, map_location=self.device))
            model.eval()
            return model
        return None

    def analyze_terrain(self, img_arr):
        h, w = img_arr.shape[:2]
        inp = cv2.resize(img_arr, (1024, 1024))
        tensor = torch.tensor(inp.transpose(2,0,1)/255.0, dtype=torch.float32).unsqueeze(0).to(self.device)

        with torch.no_grad():
            r_mask = self.road_model(tensor).squeeze().cpu().numpy()
            b_mask = self.bldg_model(tensor).squeeze().cpu().numpy()

        r_mask = cv2.resize(r_mask, (w, h))
        b_mask = cv2.resize(b_mask, (w, h))

        r_bin = closing((r_mask > 0.4).astype(np.uint8), square(3))
        b_bin = (b_mask > 0.4).astype(np.uint8)
        return r_bin, b_bin

    def calculate_tactical_routes(self, r_mask, b_mask, user_obstacles, v_name, start, end, res=0.55):
        req_w = self.vehicles.get(v_name, 3.0) + 0.5
        dist_map = distance_transform_edt(r_mask)
        width_map = dist_map * 2 * res

        # 1. 기본 기동 가능 영역 (폭 기준)
        passable = (width_map >= req_w).astype(np.uint8)

        # 2. 정적 장애물 (건물) 적용
        passable[b_mask == 1] = 0

        # 3. [New] 동적 장애물 (사용자 클릭) 적용
        # 사용자가 찍은 좌표에 반지름 20px(약 10m)짜리 원을 그려서 길을 끊어버림
        obstacle_map = np.zeros_like(passable)
        for obs in user_obstacles:
            # 원 그리기 (반경 20px)
            cv2.circle(passable, obs, 20, 0, -1) # passable에서 0(불가)으로 지움
            cv2.circle(obstacle_map, obs, 20, 1, -1) # 장애물 맵에 표시

        # 시작/끝점 보정
        start = self._snap_to_road(start, passable)
        end = self._snap_to_road(end, passable)

        if not start: return None, "출발지가 장애물 지역이거나 도로 밖입니다."
        if not end: return None, "도착지가 장애물 지역이거나 도로 밖입니다."

        # 비용 설정
        BASE_COST = 1.0
        OFFROAD_COST = 500.0

        # ① 최단 경로
        cost_short = np.where(passable == 1, BASE_COST, OFFROAD_COST)

        # ② 안전 경로 (중앙선 선호)
        safe_weight = dist_map.copy()
        cost_opt_base = 1.0 + (5.0 / (safe_weight + 1e-5))
        cost_opt = np.where(passable == 1, cost_opt_base, OFFROAD_COST)

        # 공통: 건물 및 사용자 장애물 지역은 절대 진입 불가 (Infinite Cost)
        total_block = (b_mask == 1) | (obstacle_map == 1)
        cost_short[total_block] = np.inf
        cost_opt[total_block] = np.inf

        def solve_mcp(cost_grid):
            mcp = MCP_Geometric(cost_grid)
            try:
                mcp.find_costs(starts=[(start[1], start[0])], ends=[(end[1], end[0])])
                path = mcp.traceback(end=(end[1], end[0]))
                return [(p[1], p[0]) for p in path]
            except: return None

        print("⚡ 경로 재설정 중 (장애물 회피)...")
        p1 = solve_mcp(cost_short)
        p2 = solve_mcp(cost_opt)

        # ③ 우회 경로 (최적 경로 기반 페널티)
        p3 = None
        if p2:
            cost_detour = cost_opt.copy()
            self._apply_penalty(cost_detour, p2)
            p3 = solve_mcp(cost_detour)

        if not p1 and not p2 and not p3:
            return None, "장애물로 인해 모든 경로가 차단되었습니다."

        # 거리 계산
        results = []
        if p1: results.append({"type": "최단 경로", "path": p1, "color": "blue", "style": ":", "dist": self.calc_dist(p1, res)})
        if p2: results.append({"type": "최적 경로", "path": p2, "color": "#00FF00", "style": "-", "dist": self.calc_dist(p2, res)})
        if p3: results.append({"type": "우회 경로", "path": p3, "color": "orange", "style": "--", "dist": self.calc_dist(p3, res)})

        return {"routes": results, "start": start, "end": end, "obstacles": user_obstacles}, "성공"

    def _apply_penalty(self, cost_map, path):
        path_mask = np.zeros_like(cost_map, dtype=np.uint8)
        for px, py in path:
            if 0 <= py < path_mask.shape[0] and 0 <= px < path_mask.shape[1]:
                path_mask[py, px] = 1
        penalty_zone = dilation(path_mask, disk(15))
        cost_map[penalty_zone == 1] *= 10.0

    def calc_dist(self, path, res):
        if not path or len(path) < 2: return 0.0
        dist = 0.0
        for i in range(len(path)-1):
            dist += np.sqrt((path[i][0]-path[i+1][0])**2 + (path[i][1]-path[i+1][1])**2)
        return dist * res

    def _snap_to_road(self, pt, mask, search_range=50):
        if mask[pt[1], pt[0]] == 1: return pt
        y_min = max(0, pt[1]-search_range); y_max = min(mask.shape[0], pt[1]+search_range)
        x_min = max(0, pt[0]-search_range); x_max = min(mask.shape[1], pt[0]+search_range)
        sub_mask = mask[y_min:y_max, x_min:x_max]
        y_idxs, x_idxs = np.where(sub_mask == 1)
        if len(y_idxs) == 0: return None
        y_idxs += y_min; x_idxs += x_min
        dists = (y_idxs - pt[1])**2 + (x_idxs - pt[0])**2
        idx = np.argmin(dists)
        return (x_idxs[idx], y_idxs[idx])

# 3. UI 함수들
def get_multi_click_ui(img_arr, title="지점을 클릭하세요"):
    # 이미지를 Base64로 변환
    img = Image.fromarray(img_arr)
    b = io.BytesIO(); img.save(b, format='PNG')
    b64 = base64.b64encode(b.getvalue()).decode('utf-8')

    html = f"""
    <div style="text-align:center">
        <h3 style="color:red">{title}</h3>
        <p>원하는 곳을 클릭하여 장애물을 설정하세요. 완료되면 [설정 완료] 버튼을 누르세요.</p>
        <canvas id="cv" width="{img.width}" height="{img.height}" style="border:2px solid red;cursor:crosshair"></canvas>
        <br><br>
        <button id="btn" style="font-size:16px; padding:10px 20px; cursor:pointer;">✅ 설정 완료</button>
        <p id="log">클릭 횟수: 0</p>
    </div>
    <script>
        var c = document.getElementById('cv');
        var ctx = c.getContext('2d');
        var img = new Image();
        var points = [];

        img.src = "data:image/png;base64,{b64}";
        img.onload = function() {{ ctx.drawImage(img, 0, 0); }};

        c.onclick = function(e) {{
            var r = c.getBoundingClientRect();
            var x = Math.round(e.clientX - r.left);
            var y = Math.round(e.clientY - r.top);
            points.push([x, y]);

            // 화면에 X 표시
            ctx.strokeStyle = "red";
            ctx.lineWidth = 3;
            ctx.beginPath(); ctx.moveTo(x-10, y-10); ctx.lineTo(x+10, y+10); ctx.stroke();
            ctx.beginPath(); ctx.moveTo(x+10, y-10); ctx.lineTo(x-10, y+10); ctx.stroke();
            ctx.beginPath(); ctx.arc(x, y, 20, 0, 2 * Math.PI); ctx.stroke();

            document.getElementById("log").innerText = "클릭 횟수: " + points.length;
        }};

        var p = new Promise(resolve => {{
            document.getElementById('btn').onclick = () => {{ resolve(points); }};
        }});
    </script>
    """
    display(HTML(html))
    return eval_js("p")

def get_single_click_ui(img_arr, title="지점을 클릭하세요"):
    img = Image.fromarray(img_arr)
    b = io.BytesIO(); img.save(b, format='PNG')
    b64 = base64.b64encode(b.getvalue()).decode('utf-8')
    html = f"""<div style="text-align:center"><h3 style="color:blue">{title}</h3><canvas id="cv2" width="{img.width}" height="{img.height}" style="border:2px solid blue;cursor:crosshair"></canvas></div><script>var c=document.getElementById('cv2');var ctx=c.getContext('2d');var i=new Image();i.src="data:image/png;base64,{b64}";i.onload=function(){{ctx.drawImage(i,0,0)}};var p=new Promise(r=>{{c.onclick=e=>{{var b=c.getBoundingClientRect();r([Math.round(e.clientX-b.left),Math.round(e.clientY-b.top)])}} }});</script>"""
    display(HTML(html)); return tuple(eval_js("p"))

def show_final_map(img, r_mask, b_mask, routes, result_data):
    plt.figure(figsize=(12, 12))
    plt.imshow(img)
    plt.imshow(r_mask, cmap='Oranges', alpha=0.4, vmin=0, vmax=1)
    plt.imshow(np.ma.masked_where(b_mask==0, b_mask), cmap='Greys', alpha=0.5, vmin=0, vmax=1)

    # 장애물 표시 (X)
    for obs in result_data['obstacles']:
        plt.scatter(obs[0], obs[1], c='red', s=300, marker='X', linewidths=3, zorder=20, label='Destroyed Zone')

    # 시작/끝
    s, e = result_data['start'], result_data['end']
    plt.scatter(*s, c='yellow', s=200, marker='o', edgecolors='black', zorder=25)
    plt.scatter(*e, c='red', s=200, marker='*', edgecolors='black', zorder=25)

    for r in routes:
        px, py = zip(*r['path'])
        plt.plot(px, py, c=r['color'], lw=4, ls=r['style'], label=f"{r['type']} ({r['dist']:.0f}m)")

    plt.title(f"전술 상황도 (장애물 {len(result_data['obstacles'])}개 식별)", fontsize=15, fontweight='bold')

    # 범례 정리
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper right', framealpha=0.9)

    plt.axis('off'); plt.show()

# 4. 메인 실행 함수
def run_war_room():
    ROAD_PTH = "/content/drive/MyDrive/Colab Notebooks/Data/best_road_model_1024.pth"
    BLDG_PTH = "/content/drive/MyDrive/Colab Notebooks/Data/best_building_model_1024.pth"

    sys = CommandCenterSystem(ROAD_PTH, BLDG_PTH)

    # 1. 이미지 로드
    img, fname = load_image_robust()
    if img is None: return

    # 2. [순서 변경] 설정값을 먼저 입력받습니다 (멈춤 방지)
    print("-" * 40)
    print("⚙️ 작전 환경 설정")
    v_choice = input("   1. 장비 선택 (1:K2 흑표, 2:소형전술) [기본 1]: ")
    v = "소형전술차량" if v_choice == '2' else "K2 흑표"

    res_input = input("   2. 해상도 입력 (기본 0.55): ")
    try: res = float(res_input) if res_input.strip() else 0.55
    except: res = 0.55
    print("-" * 40)

    # 3. AI 지형 분석 시작
    print("⏳ AI 정밀 지형 분석 중... (잠시만 기다려주세요)")
    r_mask, b_mask = sys.analyze_terrain(img)
    clear_output(wait=True) # 화면 청소

    # 4. 장애물 설정 (UI)
    # 오버레이 생성
    overlay = img.copy()
    overlay[r_mask == 1] = [0, 165, 255] # 도로(주황)
    vis_img = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)

    print(f"📸 작전 구역: {fname}")
    print(f"🚜 투입 장비: {v} | 📏 해상도: {res}m/px")
    print("\n🛑 [1단계] 지도에서 '기동 불가 구역'을 클릭하세요. (완료되면 버튼 클릭)")

    # 장애물 UI 호출
    obstacles = get_multi_click_ui(vis_img, "장애물 설정 (폭격/지뢰 지점 클릭)")
    clear_output(wait=True)

    print(f"✅ 장애물 {len(obstacles)}개 설정 완료.")

    # 5. 시작/도착 설정 (UI)
    # 장애물 업데이트 (지도에 표시)
    for obs in obstacles:
        cv2.circle(vis_img, tuple(obs), 20, (255, 0, 0), -1) # 빨간 원

    try:
        print("\n🔵 [2단계] 시작 지점을 클릭하세요")
        s = get_single_click_ui(vis_img, "Start Point")
        clear_output(wait=True)

        print("\n🔴 [3단계] 도착 지점을 클릭하세요")
        e = get_single_click_ui(vis_img, "Dest Point")
        clear_output(wait=True)

        # 6. 경로 계산 및 결과
        result, msg = sys.calculate_tactical_routes(r_mask, b_mask, obstacles, v, s, e, res)

        if result:
            show_final_map(img, r_mask, b_mask, result['routes'], result)
        else:
            print(f"⛔ 작전 불가: {msg}")
            # 실패 시 상황도 출력
            plt.figure(figsize=(12,12)); plt.imshow(img)
            plt.imshow(r_mask, cmap='Oranges', alpha=0.3)
            plt.imshow(np.ma.masked_where(b_mask==0, b_mask), cmap='cool', alpha=0.5)
            for obs in obstacles:
                plt.scatter(obs[0], obs[1], c='red', s=300, marker='X')
            plt.title("경로 단절 상황도", fontsize=15); plt.axis('off'); plt.show()

    except Exception as ex: print(f"오류 발생: {ex}")


def set_output_font_size(size=35):
  display(HTML(f'''
  <style>
    pre {{
      font-size: {size}px !important;
    }}
    .output_text pre {{
      font-size: {size}px !important;
    }}
  </style>
  '''))

# 원하는 폰트 크기(픽셀)를 입력하세요
set_output_font_size(35)


# 실행
run_war_room()